# COBRApy

COBRApy is a package for constraint-based modeling of biological networks written in Python.

This tool allows loading and inspecting Genome-Scale Metabolic (GEM) models written in the Systems Biology Markup Language (SBML) format.

Using COBRApy, one can analyze the following model contents:
1. Reactions
2. Metabolites
3. Genes
4. Exchange reactions (Environmental Conditions)

COBRApy allows manipulating the contents of a GEM model. For instance, one can edit reactions' flux bounds, knock out a metabolic gene, or change environmental conditions.

COBRApy contains flux analysis methods to simulate an organism's phenotypic behavior. These include Flux Balance Analysis (FBA), Parsimonious FBA, or Flux Variability Analysis (FVA).

The simulation of gene and reaction deletions for a given GEM model is a straightforward process. One can simulate single or double knockouts using one of the flux analysis methods mentioned above.

## Installation


### Requirements
The following requirements are needed to use COBRApy:
- Python 3.6 or higher
- pip must be installed
- GLPK is the default solver, but CPLEX is preferred


### How to install COBRApy?
```
pip install cobra
```

# Exercise 5 - Phenotype prediction

## Working with a GEM model

For this practical session, we will be using the following model:
- _E. coli_ core model which contains the central carbon metabolism of _Escherichia coli_ -> file: ../data/e_coli_core.xml

You can read more about _E. coli_ core model (Orth et al., 2010) in the following links:
- https://journals.asm.org/doi/10.1128/ecosalplus.10.2.1
- http://bigg.ucsd.edu/models/e_coli_core

This exercise consists of exploring the phenotype prediction tools of COBRApy. Thus, the following steps will be followed:
- Perform an FBA simulation using an aerobic/anaerobic conditions;
- Perform an pFBA simulation;
- Perform an FVA simulation;
- Perform reaction and gene deletions;
- Perform mutant-specific simulation methods, such as ROOM and MOMA;
- Perform single and double deletions
- Perform production envelopes

In [ ]:
# imports
import cobra
import escher

In [ ]:
# Loading a model
model_path = '../data/e_coli_core.xml'
model = cobra.io.read_sbml_model(model_path)

model

## Phenotype Prediction

COBRApy includes different algorithms for phenotype prediction. These include:
- Flux Balance Analysis (FBA); 
- Parsimonious Flux Balance Analysis (pFBA);
- Flux Variability Analysis (FVA);
- Regulatory On/Off Minimization (ROOM);
- Minimization of Metabolic Adjustment (MOMA);

### Flux Balance Analysis (FBA) - Aerobiosis

First, the exchange reactions should be verified to make sure that the aerobic conditions are all set up in the model.

In [ ]:
#inspecting the exchange reactions.
for exchange in model.exchanges:
    print(exchange.name, '->', exchange.bounds)

In [ ]:
# performing a FBA simulation in aerobiosis
model.summary()

### Flux Balance Analysis (FBA) - Anaerobiosis

Now, we should alter the exchange reactions to anaerobic conditions.

In [ ]:
#setting an anaerobic medium
o2_exchange = model.exchanges.get_by_id('EX_o2_e')
o2_exchange.bounds = (0, 1000)
o2_exchange

In [ ]:
# performing a FBA simulation in anaerobiosis
model.summary()

In [ ]:
# now we should revert the model changes to aerobic conditions
o2_exchange.bounds = (-1000, 1000)
o2_exchange

### Making reversible changes in the model

All changes performed in a GEM model using CORBApy are irreversible by default.
That is, if we change the bounds of the oxygen exchange reaction, our model will no longer continue under aerobic conditions during this exercise.

However, there is a way to perform reversible changes in a GEM model using COBRApy. For that, one can use the `with` context manager in our `model`. All changes performed within the `with` context manager block will be reverted automatically by COBRApy.

In [ ]:
# verifying that we have reverted the model changes to aerobic conditions
o2_exchange = model.exchanges.get_by_id('EX_o2_e')
o2_exchange

In [ ]:
# making reversible changes in the model
with model:
    o2_exchange.bounds = (0, 1000)
    model_summary = model.summary()

model_summary

In [ ]:
# the bounds of the oxygen exchange reaction have been reverted automatically this time
o2_exchange

### Flux Variability Analysis (FVA)

FBA can only obtain a unique flux distribution for a given objective function. Nevertheless, the space of flux distributions is very large and can vary significantly for a different objective. _FVA_ is a simulation method that finds the possible flux range for each reaction. _FVA_ can be used from the flux analysis package `cobra.flux_analysis.flux_variability_analysis(model)`. Note that, _FVA_ allows setting a minimum value of growth rate. In this case, we will be using 10% (that is, 0.1) of the wild-type growth rate.

In [ ]:
# performing fva simulation
fva_solution = cobra.flux_analysis.flux_variability_analysis(model, fraction_of_optimum=0.1)
fva_solution

In [ ]:
# maximum theoretical production rates of Acetate (EX_ac_e), Ethanol (EX_etoh_e), and Formate (EX_for_e)

print(f'Maximum theoretical production rate of Acetate:', fva_solution.loc['EX_ac_e', 'maximum'], 'mmol/gDW/h')
print(f'Maximum theoretical production rate of Ethanol:', fva_solution.loc['EX_etoh_e', 'maximum'], 'mmol/gDW/h')
print(f'Maximum theoretical production rate of Formate:', fva_solution.loc['EX_for_e', 'maximum'], 'mmol/gDW/h')

### Parsimonious Flux Balance Analysis (pFBA)

pFBA simulations gives the optimal growth rate, while minimizing the total sum of fluxes.
pFBA can be used from the flux analysis package `cobra.flux_analysis.pfba(model)`.

In [ ]:
#performing pfba simulation
pfba_solution = cobra.flux_analysis.pfba(model)
pfba_solution

The optimal solution of the pFBA is considerably different from the FBA result. This happens because the objective value for the pFBA is defined as the sum of all flux values (`sum(abs(pfba_solution.fluxes.values))`). On the other hand, the FBA result corresponds to the flux value of the reaction that is being optimized (`fba_solution.fluxes["BIOMASS_Ecoli_core_w_GAM"]`).

In [ ]:
#calculating the objective value of a pFBA solution
sum(abs(pfba_solution.fluxes.values))

### Simulating Deletions

As previously mentioned, COBRApy can be used to simulate gene or reaction deletions. The function `knock_out()` can be used to knock out a given reaction or gene.

In [ ]:
#knock out the SUCDi reaction
with model:
    model.reactions.SUCDi.knock_out()
    pfba_solution = cobra.flux_analysis.pfba(model)
    print('SUCDi mutant growth rate: ', pfba_solution.fluxes['BIOMASS_Ecoli_core_w_GAM'])
    print('SUCDi flux rate: ', pfba_solution.fluxes['SUCDi'])
    print('SUCDi mutant succinate production rate: ', pfba_solution.fluxes['EX_succ_e'])

In [ ]:
# knock out the b1852 gene associated with reaction G6PDH2r
with model:
    model.genes.b1852.knock_out()
    pfba_solution = cobra.flux_analysis.pfba(model)
    print('b1852 mutant growth rate: ', pfba_solution.fluxes['BIOMASS_Ecoli_core_w_GAM'])
    print('G6PDH2r flux rate: ', pfba_solution.fluxes['G6PDH2r'])
    print('b1852 mutant succinate production rate: ', pfba_solution.fluxes['EX_succ_e'])

Gene-Protein-Reaction (GPR) rules can be used to understand which genes are associated with a given reaction. Besides, one can understand by the GPR rule if the reaction is being catalyzed by a single gene, isoenzyme or protein complex. In COBRApy, one can inspect the GPR rule of a given reaction or which reactions are associated with a given gene.

In [ ]:
# ACKr GPR rule. This reaction is being catalyzed by an isoenzyme
model.reactions.ACKr.gene_reaction_rule

In [ ]:
# knock out the b3115 & b2296 & b1849 genes associated with reaction ACKr
with model:
    model.genes.b3115.knock_out()
    model.genes.b2296.knock_out()
    model.genes.b1849.knock_out()
    pfba_solution = cobra.flux_analysis.pfba(model)
    print('b3115, b2296, b1849 mutant growth rate: ', pfba_solution.fluxes['BIOMASS_Ecoli_core_w_GAM'])
    print('ACKr flux rate: ', pfba_solution.fluxes['ACKr'])
    print('b3115, b2296, b1849 mutant succinate production rate: ', pfba_solution.fluxes['EX_succ_e'])

In [ ]:
# performing all deletions at once to verify succinate production rate, EX_succ_e
with model:
    wt_pfba_solution = cobra.flux_analysis.pfba(model)
    print('WT growth rate: ', wt_pfba_solution.fluxes['BIOMASS_Ecoli_core_w_GAM'])
    print('WT succinate production rate: ', wt_pfba_solution.fluxes['EX_succ_e'])
    model.reactions.SUCDi.knock_out()
    model.genes.b1852.knock_out()
    model.genes.b3115.knock_out()
    model.genes.b2296.knock_out()
    model.genes.b1849.knock_out()
    mutant_pfba_solution = cobra.flux_analysis.pfba(model)
    print('MUTANT growth rate: ', mutant_pfba_solution.fluxes['BIOMASS_Ecoli_core_w_GAM'])
    print('MUTANT succinate production rate: ', mutant_pfba_solution.fluxes['EX_succ_e'])

In [ ]:
# WILD-TYPE flux distribution
builder = escher.Builder(map_name='e_coli_core.Core metabolism', model=model, reaction_data=wt_pfba_solution.fluxes)
builder

In [ ]:
# MUTANT flux distribution
builder = escher.Builder(map_name='e_coli_core.Core metabolism', model=model, reaction_data=mutant_pfba_solution.fluxes)
builder

### MOMA and ROOM

COBRApy includes phenotype prediction methods that are used predict the flux distribution after a gene knock out. These are the Minimization of Metabolic Adjustment (MOMA), which can be called using `cobra.flux_analysis.moma()`, and Regulatory On/Off Minimization (ROOM), using `cobra.flux_analysis.room()`.

In [ ]:
#using MOMA with COBRApy
with model:
    pfba_solution = cobra.flux_analysis.pfba(model)
    model.reactions.SUCDi.knock_out()
    model.genes.b1852.knock_out()
    model.genes.b3115.knock_out()
    model.genes.b2296.knock_out()
    model.genes.b1849.knock_out()
    moma_result = cobra.flux_analysis.moma(model, pfba_solution)
    print('MOMA Result: ', moma_result)

In [ ]:
model.summary(moma_result)

In [ ]:
#using ROOM with COBRApy
with model:
    pfba_solution = cobra.flux_analysis.pfba(model)
    model.reactions.SUCDi.knock_out()
    model.genes.b1852.knock_out()
    model.genes.b3115.knock_out()
    model.genes.b2296.knock_out()
    model.genes.b1849.knock_out()
    room_result = cobra.flux_analysis.room(model, pfba_solution)
    print('ROOM Result: ', room_result)

In [ ]:
model.summary(room_result)

### Single Deletions

Single gene and reaction deletions can also be simulated with the flux analysis package of COBRApy. To do so, one can use the `cobra.flux_analysis.single_gene_deletion()` and `cobra.flux_analysis.single_reaction_deletion()` methods.

In [ ]:
#single reaction deletion
reaction_deletion_results = cobra.flux_analysis.single_reaction_deletion(model)
reaction_deletion_results

In [ ]:
#single gene deletion
gene_deletion_results = cobra.flux_analysis.single_gene_deletion(model)
gene_deletion_results

It is worth noting that genes and reactions with a growth rate equal to zero can be considered as essential genes or essential reactions, respectively.

### Double Deletions

Double gene and reaction deletions can also be simulated with the flux analysis package of COBRApy. To do so, one can use the `cobra.flux_analysis.double_gene_deletion()` and `cobra.flux_analysis.double_reaction_deletion()` methods. These methods perform all possible combinations.

In [ ]:
#double reaction deletion
double_reaction_deletion_results = cobra.flux_analysis.double_reaction_deletion(model)
double_reaction_deletion_results

In [ ]:
#double gene deletion
double_gene_deletion_results = cobra.flux_analysis.double_gene_deletion(model)
double_gene_deletion_results